<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.4: Functional Programming
**Prev: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**<br>
**Next: [Object Oriented Programming](3.5_object_oriented_programming.ipynb)**

## Motivation
在之前的许多模块中你都看到了函数，现在是时候创建我们自己的函数并有效地使用它们了。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

本模块使用 Chisel 的 `FixedPoint` 类型，该类型目前位于实验包中。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import chisel3.experimental._
import chisel3.internal.firrtl.KnownBinaryPoint

---
# Functional Programming in Scala
Scala 函数在模块 1 中进行了介绍，你在前一个模块中也经常看到它们的使用。这里是关于函数的回顾。函数接受任意数量的输入并生成一个输出。输入通常称为函数的参数。若不生成输出，则返回 `Unit` 类型。

<span style="color:blue">**示例：自定义函数**</span><br>
下面是一些 Scala 函数的示例。

In [ ]:
// No inputs or outputs (two versions).
def hello1(): Unit = print("Hello!")
def hello2 = print("Hello again!")

// Math operation: one input and one output.
def times2(x: Int): Int = 2 * x

// Inputs can have default values, and explicitly specifying the return type is optional.
// Note that we recommend specifying the return types to avoid surprises/bugs.
def timesN(x: Int, n: Int = 2) = n * x

// Call the functions listed above.
hello1()
hello2
times2(4)
timesN(4)         // no need to specify n to use the default value
timesN(4, 3)      // argument order is the same as the order where the function was defined
timesN(n=7, x=2)  // arguments may be reordered and assigned to explicitly

## Functions as Objects
Scala 中的函数是一级对象。这意味着我们可以将一个函数赋值给一个 `val`，并将其作为参数传递给类、对象或其他函数。

<span style="color:blue">**示例：函数对象**</span><br>
下面是相同函数作为函数和作为对象实现的示例。

In [ ]:
// These are normal functions.
def plus1funct(x: Int): Int = x + 1
def times2funct(x: Int): Int = x * 2

// These are functions as vals.
// The first one explicitly specifies the return type.
val plus1val: Int => Int = x => x + 1
val times2val = (x: Int) => x * 2

// Calling both looks the same.
plus1funct(4)
plus1val(4)
plus1funct(x=4)
//plus1val(x=4)  // this doesn't work
//times2val(x=4) // this doesn't work either

为什么要创建一个 `val` 而不是 `def`？使用 `val`，你可以将函数传递给其他函数，如下所示。你甚至可以创建接受其他函数作为参数的自定义函数。正式地，接受或生成函数的函数称为*高阶函数（higher-order functions）*。你在上一个模块中看到了它们的使用，但现在你将创建自己的高阶函数！

<span style="color:blue">**示例：高阶函数**</span><br>
这里我们再次展示了 `map`，并创建了一个新函数 `opN`，它接受一个函数 `op` 作为参数。

In [ ]:
// create our function
val plus1 = (x: Int) => x + 1
val times2 = (x: Int) => x * 2

// pass it to map, a list function
val myList = List(1, 2, 5, 9)
val myListPlus = myList.map(plus1)
val myListTimes = myList.map(times2)

// create a custom function, which performs an operation on X N times using recursion
def opN(x: Int, n: Int, op: Int => Int): Int = {
  if (n <= 0) { x }
  else { opN(op(x), n-1, op) }
}

opN(7, 3, plus1)
opN(7, 3, times2)

<span style="color:blue">**Example: Functions vs. Objects**</span><br>
在使用没有参数的函数时，可能会出现令人困惑的情况。函数在每次调用时都会被求值，而 `val` 在实例化时被求值。

In [ ]:
import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function
val x = Random.nextInt
def y = Random.nextInt

// x was previously evaluated, so it is a constant
println(s"x = $x")
println(s"x = $x")

// y is a function and gets reevaluated at each call, thus these produce different results
println(s"y = $y")
println(s"y = $y")

## Anonymous Functions
顾名思义，匿名函数是没有名字的。如果我们只使用一次函数，就不需要为它创建一个 `val`。

<span style="color:blue">**示例：匿名函数**</span><br>
下面的示例演示了这一点。它们通常在作用域中（用花括号代替圆括号）。

In [ ]:
val myList = List(5, 6, 7, 8)

// add one to every item in the list using an anonymous function
// arguments get passed to the underscore variable
// these all do the same thing
myList.map( (x:Int) => x + 1 )
myList.map(_ + 1)

// a common situation is to use case statements within an anonymous function
val myAnyList = List(1, 2, "3", 4L, myList)
myAnyList.map {
  case (_:Int|_:Long) => "Number"
  case _:String => "String"
  case _ => "error"
}

<span style="color:red">**练习：序列操作**</span><br>
你将使用的一组常见高阶函数是 `scanLeft`/`scanRight`，`reduceLeft`/`reduceRight`，以及 `foldLeft`/`foldRight`。理解每个函数的工作原理以及何时使用它们非常重要。默认情况下，`scan`、`reduce` 和 `fold` 的方向是左，但这并不适用于所有情况。

In [ ]:
val exList = List(1, 5, 7, 100)

// write a custom function to add two numbers, then use reduce to find the sum of all values in exList
def add(a: Int, b: Int): Int = ???
val sum = ???

// find the sum of exList using an anonymous function (hint: you've seen this before!)
val anon_sum = ???

// find the moving average of exList from right to left using scan; make the result (ma2) a list of doubles
def avg(a: Int, b: Double): Double = ???
val ma2 = ???

In [ ]:
assert(add(88, 88) == 176)
assert(sum == 113)

assert(anon_sum == 113)

assert(avg(100, 100.0) == 100.0)
assert(ma2 == List(8.875, 16.75, 28.5, 50.0, 0.0))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def add(a: Int, b: Int): Int = a + b
val sum = exList.reduce(add)

val anon\_sum = exList.reduce(\_ + \_)

def avg(a: Int, b: Double): Double = (a + b)/2.0
val ma2 = exList.scanRight(0.0)(avg)
</pre></article></div></section></div>

---
# Functional Programming in Chisel
让我们看看在 Chisel 中创建硬件生成器时如何使用函数式编程的一些示例。

<span style="color:blue">**示例：FIR 滤波器**</span><br>
首先，我们将重新审视之前示例中的 FIR 滤波器。我们不会将系数作为参数传递给类或使它们可编程，而是将一个函数传递给 FIR，该函数定义了窗口系数的计算方式。这个函数将接受窗口长度和位宽来生成缩放后的系数列表。这里有两个示例窗口。为了避免小数，我们将系数缩放到最大和最小整数值之间。有关这些窗口的更多信息，请查看[这个维基百科页面](https://en.wikipedia.org/wiki/Window_function)。

In [ ]:
// get some math functions
import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window
val TriangularWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x:Int) => 1-abs((x.toDouble-(length-1)/2.0)/((length-1)/2.0)) )
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// Hamming window
val HammingWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x: Int) => 0.54 - 0.46*cos(2*Pi*x/(length-1)))
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// check it out! first argument is the window length, and second argument is the bitwidth
TriangularWindow(10, 16)
HammingWindow(10, 16)

现在我们将创建一个接受窗口函数作为参数的 FIR 滤波器。这使我们能够在以后定义新的窗口，并保留相同的 FIR 生成器。它还使我们能够独立地调整 FIR 的大小，知道窗口将为不同的长度或位宽重新计算。由于我们在编译时选择窗口，这些系数是固定的。

In [ ]:
// our FIR has parameterized window length, IO bitwidth, and windowing function
class MyFir(length: Int, bitwidth: Int, window: (Int, Int) => Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt((bitwidth*2+length-1).W)) // expect bit growth, conservative but lazy
  })

  // calculate the coefficients using the provided window function, mapping to UInts
  val coeffs = window(length, bitwidth).map(_.U)
  
  // create an array holding the output of the delays
  // note: we avoid using a Vec here since we don't need dynamic indexing
  // 动态索引：Vec 主要在需要通过硬件索引访问元素的情况下使用，比如寄存器文件
  // 如果不需要动态索引，而只是简单地按顺序处理元素，可以使用 Seq 或其他 Scala 集合
  val delays = Seq.fill(length)(Wire(UInt(bitwidth.W))).scan(io.in)( (prev: UInt, next: UInt) => {
    next := RegNext(prev)
    next
  })
  
  // multiply, putting result in "mults"
  val mults = delays.zip(coeffs).map{ case(delay: UInt, coeff: UInt) => delay * coeff }
  
  // add up multiplier outputs with bit growth
  val result = mults.reduce(_+&_)

  // connect output
  io.out := result
}

visualize(() => new MyFir(7, 12, TriangularWindow))

最后三行可以很容易地合并成一行。此外，注意我们如何保守地处理位宽增长以避免损失。

<span style="color:blue">**示例：FIR 滤波器测试器**</span><br>
让我们测试我们的 FIR！之前，我们提供了一个自定义的 golden model。这次我们将使用 Breeze，一个有用的线性代数和信号处理函数的 Scala 库，作为我们 FIR 滤波器的 golden model。下面的代码比较了 Chisel 输出与 golden model 输出，任何错误都会导致测试失败。

尝试取消注释 expect 调用之后的打印语句。还可以尝试将窗口从三角更改为汉明窗口。

In [ ]:
// math imports
import scala.math.{pow, sin, Pi}
import breeze.signal.{filter, OptOverhang}
import breeze.signal.support.{CanFilter, FIRKernel1D}
import breeze.linalg.DenseVector

// test parameters
val length = 7
val bitwidth = 12 // must be less than 15, otherwise Int can't represent the data, need BigInt
val window = TriangularWindow

// test our FIR
test(new MyFir(length, bitwidth, window)) { c =>
    
    // test data
    val n = 100 // input length
    val sine_freq = 10
    val samp_freq = 100

    // sample data, scale to between 0 and 2^bitwidth
    val max_value = pow(2, bitwidth)-1
    val sine = (0 until n).map(i => (max_value/2 + max_value/2*sin(2*Pi*sine_freq/samp_freq*i)).toInt)
    //println(s"input = ${sine.toArray.deep.mkString(", ")}")

    // coefficients
    val coeffs = window(length, bitwidth)
    //println(s"coeffs = ${coeffs.toArray.deep.mkString(", ")}")

    // use breeze filter as golden model; need to reverse coefficients
    val expected = filter(
        DenseVector(sine.toArray),
        FIRKernel1D(DenseVector(coeffs.reverse.toArray), 1.0, ""),
        OptOverhang.None
    )
    expected.toArray // seems to be necessary
    //println(s"exp_out = ${expected.toArray.deep.mkString(", ")}") // this seems to be necessary

    // push data through our FIR and check the result
    c.reset.poke(true.B)
    c.clock.step(5)
    c.reset.poke(false.B)
    for (i <- 0 until n) {
        c.io.in.poke(sine(i).U)
        if (i >= length-1) { // wait for all registers to be initialized since we didn't zero-pad the data
            val expectValue = expected(i-length+1)
            //println(s"expected value is $expectValue")
            c.io.out.expect(expected(i-length+1).U)
            //println(s"cycle $i, got ${c.io.out.peek()}, expect ${expected(i-length+1)}")
        }
        c.clock.step(1)
    }
}

---
# Chisel Exercises
完成以下练习，以练习编写函数，将它们作为硬件生成器的参数，并避免使用可变数据。

<span style="color:red">**练习：神经网络神经元**</span><br>
我们的第一个例子是构建一个神经元，这是人工神经网络中全连接层的构建块。神经元接受输入和一组权重，每个输入一个，并生成一个输出。权重和输入相乘并相加，结果通过激活函数。在这个练习中，你将实现不同的激活函数，并将它们作为参数传递给你的神经元生成器。

![Neuron](https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/ArtificialNeuronModel_english.png/600px-ArtificialNeuronModel_english.png)

首先，完成以下代码以创建一个神经元生成器。参数 `inputs` 给出了输入的数量。参数 `act` 是实现激活函数逻辑的函数。我们将输入和输出设为 16 位定点值，具有 8 个小数位。

In [ ]:
class Neuron(inputs: Int, act: FixedPoint => FixedPoint) extends Module {
  val io = IO(new Bundle {
    val in      = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val weights = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val out     = Output(FixedPoint(16.W, 8.BP))
  })
  
  ???
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val mac = io.in.zip(io.weights).map{ case(a:FixedPoint, b:FixedPoint) => a*b}.reduce(_+_)
  io.out := act(mac)
</pre></article></div></section></div>

现在让我们创建一些激活函数！我们将使用零作为阈值。典型的激活函数有 sigmoid 函数和修正线性单元（ReLU）。

我们使用的 sigmoid 称为[逻辑函数](https://en.wikipedia.org/wiki/Logistic_function)，其公式为：

$$
logistic(x) = \frac{1}{1 + e^{-\beta x}}
$$

其中 $\beta$ 是斜率因子。然而，在硬件中计算指数函数是相当具有挑战性且昂贵的。我们将其近似为阶跃函数。

$$
step(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             1  & \text{if } x \gt 0
       \end{cases}
$$

第二个函数 ReLU 的公式类似。

$$
relu(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             x  & \text{if } x \gt 0
       \end{cases}
$$

在下面实现这两个函数。你可以像这样指定一个定点字面量：`-3.14.F(8.BP)`。

In [ ]:
val Step: FixedPoint => FixedPoint = ???
val ReLU: FixedPoint => FixedPoint = ???

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val Step: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), 1.F(8.BP))
val ReLU: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), x)
</pre></article></div></section></div>

最后，让我们创建一个测试器来检查我们神经元的正确性。使用阶跃激活函数时，神经元可以用作逻辑门的近似器。选择适当的权重和偏差可以执行二进制函数。我们将使用 AND 逻辑来测试我们的神经元。完成以下测试器以使用阶跃函数检查我们的神经元。

注意，由于电路是纯组合逻辑的，因此不需要调用 `reset(5)` 和 `step(1)`。

In [ ]:
// test our Neuron 
test(new Neuron(2, Step)) { c =>
    val inputs = Seq(Seq(-1, -1), Seq(-1, 1), Seq(1, -1), Seq(1, 1))

    // make this a sequence of two values
    val weights = ???

    // push data through our Neuron and check the result (AND gate)
    c.reset.poke(true.B)
    c.clock.step(5)
    c.reset.poke(false.B)
    for (i <- inputs) {
        c.io.in(0).poke(i(0).F(8.BP))
        c.io.in(1).poke(i(1).F(8.BP))
        c.io.weights(0).poke(weights(0).F(16.W, 8.BP))
        c.io.weights(1).poke(weights(1).F(16.W, 8.BP))
        c.io.out.expect((if (i(0) + i(1) > 0) 1 else 0).F(16.W, 8.BP))
        c.clock.step(1)
    }
    
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val weights  = Seq(1.0, 1.0)
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)